In [13]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# === Last inn data ===
df = pd.read_csv("data/processed/summary_statistics.csv", sep=";")
df.columns = df.columns.str.strip()

# Konverter dato og trekk ut måned
df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
df = df.dropna(subset=['Dato'])
df['måned'] = df['Dato'].dt.to_period('M')

#Fra 2017-2024
#df = df[(df['Dato'] >= "2017-01-01") & (df['Dato'] <= "2024-12-31")]

# Lag lagget rensefiskandel (per PO)
df = df.sort_values(['PO', 'måned'])
df['% Rensefisk_lag1'] = df.groupby('PO')['% Rensefisk'].shift(1)

# Kvadrert temperatur
df["Temperatur2"] = df["Temperature"]**2

# Drop rader med NaN i nødvendige kolonner
relevante_kolonner = [
    "% Dødfisk", "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK",
    "% Rensefisk_lag1", "badebehandling", "mekanisk behandling", "termisk behandling", "PO"
]
df = df.dropna(subset=relevante_kolonner)

# Antall observasjoner etter filtrering
print(f"Antall observasjoner brukt i analysen: {len(df)}")

# === Definer modeller ===
f1 = 'Q("% Dødfisk") ~ Q("Temperature") + C(PO)'
f2 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + C(PO)'
f3 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + C(PO)'
f4 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + Q("% Rensefisk_lag1") + Q("badebehandling") + Q("mekanisk behandling") + Q("termisk behandling") + C(PO)'

# === Estimer modeller ===
model1 = smf.ols(f1, data=df).fit(cov_type='HC3')
model2 = smf.ols(f2, data=df).fit(cov_type='HC3')
model3 = smf.ols(f3, data=df).fit(cov_type='HC3')
model4 = smf.ols(f4, data=df).fit(cov_type='HC3')

# Antall observasjoner per modell
print(f"Observasjoner i modell 1: {int(model1.nobs)}")
print(f"Observasjoner i modell 2: {int(model2.nobs)}")
print(f"Observasjoner i modell 3: {int(model3.nobs)}")
print(f"Observasjoner i modell 4: {int(model4.nobs)}")

# === Lag regresjonstabell ===
summary = summary_col(
    [model1, model2, model3, model4],
    stars=True,
    float_format='%0.4f',
    model_names=["(1)", "(2)", "(3)", "(4)"],
    info_dict={
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'N': lambda x: f"{int(x.nobs)}"
    },
    regressor_order=[
        'Intercept',
        'Q("Temperature")',
        'Q("Temperatur2")',
        'Q("Tetthet")',
        'Q("BA_ano")',
        'Q("% FORFORBRUK")',
        'Q("% Rensefisk_lag1")',
        'Q("badebehandling")',
        'Q("mekanisk behandling")',
        'Q("termisk behandling")',
    ]
)

# === Vis resultat ===
print(summary)
# === Lagre som LaTeX-fil ===
#with open("results/regresjonsresultater_2012_2024.tex", "w") as f:
    #f.write(summary.as_latex())


Antall observasjoner brukt i analysen: 1813
Observasjoner i modell 1: 1813
Observasjoner i modell 2: 1813
Observasjoner i modell 3: 1813
Observasjoner i modell 4: 1813

                            (1)        (2)        (3)        (4)    
--------------------------------------------------------------------
Intercept                0.8474***  0.8975***  1.0392***  1.0387*** 
                         (0.0512)   (0.0530)   (0.0749)   (0.0757)  
Q("Temperature")         0.0091***  0.0650***  0.0103     0.0182    
                         (0.0028)   (0.0070)   (0.0193)   (0.0193)  
Q("Temperatur2")                               0.0023***  0.0019**  
                                               (0.0008)   (0.0008)  
Q("Tetthet")                        0.0273***  0.0286***  0.0248*** 
                                    (0.0052)   (0.0052)   (0.0052)  
Q("BA_ano")                         -0.0346*** -0.0273*** -0.0239** 
                                    (0.0097)   (0.0099)   (0.0100)  
Q("

Fra 2017

In [5]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# === Last inn datasett for kort periode ===
df = pd.read_csv("data/processed/summary_statistics_2017.csv", sep=";")
df.columns = df.columns.str.strip()

# Konverter dato og trekk ut måned
df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
df = df.dropna(subset=['Dato'])
df['måned'] = df['Dato'].dt.to_period('M')

# Lag lagget rensefiskandel (per PO)
df = df.sort_values(['PO', 'måned'])
df['% Rensefisk_lag1'] = df.groupby('PO')['% Rensefisk'].shift(1)

# Kvadrert temperatur
df["Temperatur2"] = df["Temperature"]**2

# Fjern rader med manglende verdier
relevante_kolonner = [
    "% Dødfisk", "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK",
    "% Rensefisk_lag1", "badebehandling", "mekanisk behandling", "termisk behandling", "PO"
]
df = df.dropna(subset=relevante_kolonner)

# Skriv ut antall observasjoner
print(f"Antall observasjoner brukt i analysen (2017–2024): {len(df)}")

# === Definer modeller ===
f1 = 'Q("% Dødfisk") ~ Q("Temperature") + C(PO)'
f2 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + C(PO)'
f3 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + C(PO)'
f4 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + Q("% Rensefisk_lag1") + Q("badebehandling") + Q("mekanisk behandling") + Q("termisk behandling") + C(PO)'

# === Estimer modeller ===
model1 = smf.ols(f1, data=df).fit(cov_type='HC3')
model2 = smf.ols(f2, data=df).fit(cov_type='HC3')
model3 = smf.ols(f3, data=df).fit(cov_type='HC3')
model4 = smf.ols(f4, data=df).fit(cov_type='HC3')

# === Lag regresjonstabell ===
summary = summary_col(
    [model1, model2, model3, model4],
    stars=True,
    float_format='%0.4f',
    model_names=["(1)", "(2)", "(3)", "(4)"],
    info_dict={
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'N': lambda x: f"{int(x.nobs)}"
    },
    regressor_order=[
        'Intercept',
        'Q("Temperature")',
        'Q("Temperatur2")',
        'Q("Tetthet")',
        'Q("BA_ano")',
        'Q("% FORFORBRUK")',
        'Q("% Rensefisk_lag1")',
        'Q("badebehandling")',
        'Q("mekanisk behandling")',
        'Q("termisk behandling")',
    ]
)

# === Vis resultater ===
print(summary)

# === Lagre LaTeX-tabell (valgfritt) ===
#with open("results/regresjonsresultater_2017_2024.tex", "w") as f:
   #f.write(summary.as_latex())


Antall observasjoner brukt i analysen (2017–2024): 947

                            (1)        (2)        (3)        (4)    
--------------------------------------------------------------------
Intercept                0.8750***  1.1865***  1.0579***  1.0539*** 
                         (0.0790)   (0.1210)   (0.1119)   (0.1107)  
Q("Temperature")         -0.0160*** -0.0991*** 0.0022     0.0025    
                         (0.0039)   (0.0203)   (0.0289)   (0.0286)  
Q("Temperatur2")                    0.0042***  0.0014     0.0014    
                                    (0.0010)   (0.0011)   (0.0011)  
Q("Tetthet")                                   0.0408***  0.0347*** 
                                               (0.0088)   (0.0086)  
Q("BA_ano")                                    -0.0364**  -0.0282** 
                                               (0.0144)   (0.0144)  
Q("% FORFORBRUK")                              -0.0310*** -0.0328***
                                               

Nærmest Eirik sin: 

In [35]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# === Last inn data ===
df = pd.read_csv("data/processed/summary_statistics_2017.csv", sep=";")
df.columns = df.columns.str.strip()

# Konverter dato og trekk ut måned
df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
df = df.dropna(subset=['Dato'])
df['måned'] = df['Dato'].dt.to_period('M')

# Lag lagget rensefiskandel (per PO)
df = df.sort_values(['PO', 'måned'])
df['% Rensefisk_lag1'] = df.groupby('PO')['% Rensefisk'].shift(1)

# Kvadrert temperatur
df["Temperatur2"] = df["Temperature"]**2

# Drop rader med NaN i nødvendige kolonner
relevante_kolonner = [
    "% Dødfisk", "Temperature", "Temperatur2", "Tetthet", "BA_ano",
    "% FORFORBRUK", "% Rensefisk_lag1", "badebehandling", 
    "mekanisk behandling", "termisk behandling", "PO"
]
df = df.dropna(subset=relevante_kolonner)

# === Definer modeller ===
f1 = 'Q("% Dødfisk") ~ Q("Temperature") + C(PO)'
f2 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + C(PO)'
f3 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + C(PO)'
f4 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + Q("% Rensefisk_lag1") + Q("badebehandling") + Q("mekanisk behandling") + Q("termisk behandling") + C(PO)'
# === Estimer modeller ===
model1 = smf.ols(f1, data=df).fit(cov_type='HC3')
model2 = smf.ols(f2, data=df).fit(cov_type='HC3')
model3 = smf.ols(f3, data=df).fit(cov_type='HC3')
model4 = smf.ols(f4, data=df).fit(cov_type='HC3')

# === Lag regresjonstabell ===
summary = summary_col(
    [model1, model2, model3, model4],
    stars=True,
    float_format='%0.4f',
    model_names=["(1)", "(2)", "(3)", "(4)"],
    info_dict={
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'N': lambda x: f"{int(x.nobs)}"
    },
    regressor_order=[
        'Intercept',
        'Q("Temperature")',
        'Q("Temperatur2")',
        'Q("Tetthet")',
        'Q("BA_ano")',
        'Q("% FORFORBRUK")',
        'Q("% Rensefisk_lag1")',
        'Q("badebehandling")',
        'Q("mekanisk behandling")',
        'Q("termisk behandling")',
    ]
)

# === Print resultater ===
print(summary)

df_clean = df.dropna(subset=relevante_kolonner)
print(len(df_clean))



                            (1)        (2)        (3)        (4)    
--------------------------------------------------------------------
Intercept                0.8750***  1.1865***  1.0579***  1.0539*** 
                         (0.0790)   (0.1210)   (0.1119)   (0.1107)  
Q("Temperature")         -0.0160*** -0.0991*** 0.0022     0.0025    
                         (0.0039)   (0.0203)   (0.0289)   (0.0286)  
Q("Temperatur2")                    0.0042***  0.0014     0.0014    
                                    (0.0010)   (0.0011)   (0.0011)  
Q("Tetthet")                                   0.0408***  0.0347*** 
                                               (0.0088)   (0.0086)  
Q("BA_ano")                                    -0.0364**  -0.0282** 
                                               (0.0144)   (0.0144)  
Q("% FORFORBRUK")                              -0.0310*** -0.0328***
                                               (0.0059)   (0.0059)  
Q("% Rensefisk_lag1")            

Test for multilinæritet:

In [9]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Last inn datasett ===
df_all = pd.read_csv("data/processed/summary_statistics.csv", sep=";")
df_short = pd.read_csv("data/processed/summary_statistics_2017.csv", sep=";")

# === Funksjon for å klargjøre data og beregne VIF ===
def kjør_vif_analysis(df):
    df.columns = df.columns.str.strip()
    df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
    df = df.dropna(subset=['Dato'])
    df['måned'] = df['Dato'].dt.to_period('M')
    df = df.sort_values(['PO', 'måned'])
    df['% Rensefisk_lag1'] = df.groupby('PO')['% Rensefisk'].shift(1)
    df["Temperatur2"] = df["Temperature"] ** 2

    relevante_kolonner = [
        "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK",
        "% Rensefisk_lag1", "badebehandling", "mekanisk behandling", "termisk behandling"
    ]
    df_vif = df[relevante_kolonner].dropna()

    vif_df = pd.DataFrame()
    vif_df["Variabel"] = df_vif.columns
    vif_df["VIF"] = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]
    return vif_df

# === Filter for 2017–2024 for kort horisont ===
df_short['Dato'] = pd.to_datetime(df_short['Dato'], errors='coerce')
df_short_filtered = df_short[(df_short['Dato'] >= "2017-01-01") & (df_short['Dato'] <= "2024-12-31")]

# === Kjør VIF for begge datasett ===
vif_lang = kjør_vif_analysis(df_all)
vif_kort = kjør_vif_analysis(df_short_filtered)

# === Skriv ut resultatene ===
print("\nVIF for lang horisont (2012–2024):")
print(vif_lang)

print("\nVIF for kort horisont (2017–2024):")
print(vif_kort)



VIF for lang horisont (2012–2024):
              Variabel         VIF
0          Temperature  175.877983
1          Temperatur2   36.335632
2              Tetthet    4.796042
3               BA_ano    1.377613
4         % FORFORBRUK   64.600703
5     % Rensefisk_lag1    1.554819
6       badebehandling    2.908668
7  mekanisk behandling    2.892658
8   termisk behandling    2.871613

VIF for kort horisont (2017–2024):
              Variabel         VIF
0          Temperature  165.353880
1          Temperatur2   35.778397
2              Tetthet    4.934999
3               BA_ano    1.372747
4         % FORFORBRUK   59.677139
5     % Rensefisk_lag1    1.709830
6       badebehandling    2.294902
7  mekanisk behandling    2.933623
8   termisk behandling    2.907976


/var/folders/bq/c32p7bj504jfbvsz2xcgkl1m0000gn/T/ipykernel_51350/3929492010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')


In [20]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Last inn datasett ===
df_all = pd.read_csv("data/processed/summary_statistics.csv", sep=";")
df_short = pd.read_csv("data/processed/summary_statistics.csv", sep=";")

# === Forbered data ===
def klargjør_data(df):
    df.columns = df.columns.str.strip()
    df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
    df = df.dropna(subset=['Dato'])
    df['måned'] = df['Dato'].dt.to_period('M')
    df = df.sort_values(['PO', 'måned'])
    df['Temperatur2'] = df["Temperature"] ** 2
    return df

df_all = klargjør_data(df_all)
df_short = klargjør_data(df_short)
df_short_filtered = df_short[(df_short['Dato'] >= "2017-01-01") & (df_short['Dato'] <= "2024-12-31")]

# === Kjør VIF-analyse (modell f3 uten lakselus-variabler) ===
def kjør_vif_analysis(df):
    relevante_kolonner = [
        "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK"
    ]
    df_vif = df[relevante_kolonner].dropna()
    vif_df = pd.DataFrame()
    vif_df["Variabel"] = df_vif.columns
    vif_df["VIF"] = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]
    return vif_df

vif_resultat = kjør_vif_analysis(df_short_filtered)
print("\nVIF-resultat for modell f3 (2017–2024):")
print(vif_resultat)

# === Kjør OLS-modell f3 ===
formel_f3 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + Q("Tetthet") + Q("BA_ano") + Q("% FORFORBRUK") + C(PO)'
df_model = df_short_filtered.dropna(subset=[
    "% Dødfisk", "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK", "PO"
])
modell_f3 = smf.ols(formula=formel_f3, data=df_model).fit(cov_type='HC3')

# === Vis modellresultater ===
print("\nResultater for OLS-modell f3 (2017–2024):")
print(modell_f3.summary())



VIF-resultat for modell f3 (2017–2024):
       Variabel         VIF
0   Temperature  162.829732
1   Temperatur2   33.866052
2       Tetthet    4.006669
3        BA_ano    1.366368
4  % FORFORBRUK   58.726704

Resultater for OLS-modell f3 (2017–2024):
                            OLS Regression Results                            
Dep. Variable:         Q("% Dødfisk")   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     66.47
Date:                Sun, 04 May 2025   Prob (F-statistic):          1.24e-155
Time:                        16:12:02   Log-Likelihood:                -363.50
No. Observations:                1126   AIC:                             763.0
Df Residuals:                    1108   BIC:                             853.5
Df Model:                          17                                         
Covariance Type:                  HC3

In [19]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === Last inn datasett ===
df_short = pd.read_csv("data/processed/summary_statistics.csv", sep=";")

# === Forbered data ===
def klargjør_data(df):
    df.columns = df.columns.str.strip()
    df['Dato'] = pd.to_datetime(df['Dato'], errors='coerce')
    df = df.dropna(subset=['Dato'])
    df['måned'] = df['Dato'].dt.to_period('M')
    df = df.sort_values(['PO', 'måned'])
    df['Temperatur2'] = df["Temperature"] ** 2
    return df

df_short = klargjør_data(df_short)
df_short_filtered = df_short[(df_short['Dato'] >= "2017-01-01") & (df_short['Dato'] <= "2024-12-31")]

# === Kjør VIF-analyse (modell f2) ===
def kjør_vif_analysis(df):
    relevante_kolonner = ["Temperature", "Temperatur2"]
    df_vif = df[relevante_kolonner].dropna()
    vif_df = pd.DataFrame()
    vif_df["Variabel"] = df_vif.columns
    vif_df["VIF"] = [variance_inflation_factor(df_vif.values, i) for i in range(df_vif.shape[1])]
    return vif_df

vif_resultat = kjør_vif_analysis(df_short_filtered)
print("\nVIF-resultat for modell f2 (2017–2024):")
print(vif_resultat)

# === Kjør OLS-modell f2 ===
formel_f2 = 'Q("% Dødfisk") ~ Q("Temperature") + Q("Temperatur2") + C(PO)'
df_model = df_short_filtered.dropna(subset=["% Dødfisk", "Temperature", "Temperatur2", "PO"])
modell_f2 = smf.ols(formula=formel_f2, data=df_model).fit(cov_type='HC3')

# === Vis modellresultater ===
print("\nResultater for OLS-modell f2 (2017–2024):")
print(modell_f2.summary())



VIF-resultat for modell f2 (2017–2024):
      Variabel        VIF
0  Temperature  12.196633
1  Temperatur2  12.196633

Resultater for OLS-modell f2 (2017–2024):
                            OLS Regression Results                            
Dep. Variable:         Q("% Dødfisk")   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     71.81
Date:                Sun, 04 May 2025   Prob (F-statistic):          1.69e-144
Time:                        16:11:14   Log-Likelihood:                -410.04
No. Observations:                1126   AIC:                             850.1
Df Residuals:                    1111   BIC:                             925.5
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                       coef    std err          

In [21]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan

# === Last inn og klargjør data ===
df = pd.read_csv("data/processed/summary_statistics.csv", sep=";")
df.columns = df.columns.str.strip()
df["Dato"] = pd.to_datetime(df["Dato"], errors="coerce")
df = df.dropna(subset=["Dato"])
df["måned"] = df["Dato"].dt.to_period("M")
df = df.sort_values(["PO", "måned"])
df["% Rensefisk_lag1"] = df.groupby("PO")["% Rensefisk"].shift(1)
df["Temperatur2"] = df["Temperature"]**2

# === Rens data for Modell 4 ===
relevante_kolonner = [
    "% Dødfisk", "Temperature", "Temperatur2", "Tetthet", "BA_ano", "% FORFORBRUK",
    "% Rensefisk_lag1", "badebehandling", "mekanisk behandling", "termisk behandling", "PO"
]
df = df.dropna(subset=relevante_kolonner)

# === Modell 4 med PO-fixed effects ===
formel = '`% Dødfisk` ~ Temperature + Temperatur2 + Tetthet + BA_ano + Q("% FORFORBRUK") + Q("% Rensefisk_lag1") + badebehandling + Q("mekanisk behandling") + Q("termisk behandling") + C(PO)'
model = smf.ols(formel, data=df).fit()

# === Breusch–Pagan-test ===
from statsmodels.stats.diagnostic import het_breuschpagan
bp_test = het_breuschpagan(model.resid, model.model.exog)
bp_results = {
    "LM-statistikk": bp_test[0],
    "LM p-verdi": bp_test[1],
    "F-statistikk": bp_test[2],
    "F p-verdi": bp_test[3]
}

# === Skriv ut resultater ===
print("Breusch–Pagan-test (med PO):")
for navn, verdi in bp_results.items():
    print(f"{navn}: {verdi:.4f}")


PatsyError: error tokenizing input (maybe an unclosed string?)
    `% Dødfisk` ~ Temperature + Temperatur2 + Tetthet + BA_ano + Q("% FORFORBRUK") + Q("% Rensefisk_lag1") + badebehandling + Q("mekanisk behandling") + Q("termisk behandling") + C(PO)
    ^

In [32]:
import pandas as pd

# Steg 1: Les inn dataene
summary_statistics = pd.read_csv('data/processed/summary_statistics.csv', sep=';')
summary_statistics_2017 = pd.read_csv('data/processed/summary_statistics_2017.csv', sep=';')

# Steg 2: Standardiser kolonnenavn
rename_map = {
    "% Dødfisk": "dødfisk_pct",
    "Temperature": "temp",
    "Temperatur2": "temp_sq",  # I tilfelle kolonnen faktisk finnes
    "Tetthet": "tett_massekm2",
    "BA_ano": "ba_ano",
    "% FORFORBRUK": "for_pct",
    "% Rensefisk_lag1": "rensefisk_pct",
    "badebehandling": "badebeh",
    "PO": "PO"
}

summary_statistics.rename(columns=rename_map, inplace=True)
summary_statistics_2017.rename(columns=rename_map, inplace=True)

# Steg 3: Lag temp_sq manuelt hvis ikke tilstede
def ensure_temp_sq(df):
    if "temp" in df.columns and "temp_sq" not in df.columns:
        df["temp_sq"] = df["temp"] ** 2

ensure_temp_sq(summary_statistics)
ensure_temp_sq(summary_statistics_2017)

# Steg 4: Definer modellvariabler
variables_model = [
    ["dødfisk_pct", "temp", "tett_massekm2", "ba_ano", "for_pct", "PO"],
]

# Steg 5: Lag korrelasjonsmatriser og samle dem i én DataFrame
combined_df = pd.DataFrame()
for i, dataset in enumerate([summary_statistics, summary_statistics_2017]):
    horizon = "Lang_horisont" if i == 0 else "Kort_horisont"
    for j, vars_model in enumerate(variables_model):
        numeric_vars = [var for var in vars_model if var != "PO" and var in dataset.columns]
        corr_matrix = dataset[numeric_vars].corr().round(3)
        corr_matrix.insert(0, 'Variable', corr_matrix.index)
        corr_matrix['Modell'] = f"{horizon}_Modell_{j+1 + i*4}"
        combined_df = pd.concat([combined_df, corr_matrix], ignore_index=True)

# Lagre alt i én CSV-fil
combined_df.to_csv("alle_korrelasjonsmatriser.csv", index=False)

# Vis til terminal
print(combined_df.head(20))


        Variable  dødfisk_pct   temp  tett_massekm2  ba_ano  for_pct  \
0    dødfisk_pct        1.000  0.186          0.292   0.014    0.008   
1           temp        0.186  1.000          0.134   0.457    0.870   
2  tett_massekm2        0.292  0.134          1.000  -0.033    0.090   
3         ba_ano        0.014  0.457         -0.033   1.000    0.314   
4        for_pct        0.008  0.870          0.090   0.314    1.000   
5    dødfisk_pct        1.000 -0.015          0.365  -0.109   -0.142   
6           temp       -0.015  1.000          0.165   0.459    0.847   
7  tett_massekm2        0.365  0.165          1.000  -0.029    0.109   
8         ba_ano       -0.109  0.459         -0.029   1.000    0.294   
9        for_pct       -0.142  0.847          0.109   0.294    1.000   

                   Modell  
0  Lang_horisont_Modell_1  
1  Lang_horisont_Modell_1  
2  Lang_horisont_Modell_1  
3  Lang_horisont_Modell_1  
4  Lang_horisont_Modell_1  
5  Kort_horisont_Modell_5  
6  Kort_hor